In [ ]:
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
large = 30
med = 20
small = 15
params = {"axes.titlesize": med,
"axes.titlepad": med,
"legend.fontsize": med, 
"axes.labelsize": large,
"axes.titlesize": med,
"xtick.labelsize": large,
"ytick.labelsize": large,
"figure.titlesize": med}
plt.rcParams["font.family"] = "Helvetica"
plt.rcParams["font.serif"] = ["Helvetica Neue"]
plt.rcParams.update(params)

# Loading the fermionic Hamiltonian into the program

In [ ]:
from VQA import VQA

with open("solvated_active_second_q.pickle", "rb") as f:
    fermionic_hamiltonian = pickle.load(f)
vqa = VQA(fermionic_hamiltonian, 1.e-3, True)

# Expectation value of the Hamiltonian

In [ ]:
gamma_in = 1.0
gamma_out = 1.0
dt = 0.8
number_of_data_points = 3

observable_expectation_value_vs_time_avg, observable_expectation_value_vs_time_std, circuit_lst, statevector_lst = vqa.energy_expectation_value_estimator(
     gamma_in = gamma_in,
     gamma_out = gamma_out,
     dt = dt,
     number_of_data_points = number_of_data_points,
     simulation_type = {"Noisy": True, "T_1": 500e3, "T_2": 450e3}
)

# Plot of energy as function of time

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

time_lst = np.arange(0, dt * number_of_data_points, dt)
x = time_lst[:number_of_data_points - 1]
y = observable_expectation_value_vs_time_avg
plt.plot(x, y, marker='o', label='Energy', color='rebeccapurple', markersize=20, linewidth=2.5)
ax.set_xlabel(r"Time")
ax.set_ylabel(r"Energy", labelpad = 10)
for spine in ax.spines.values():
     spine.set_linewidth(2)
ax.tick_params(width=2, length=10, direction='in', which='major')
plt.show()

# Plot of Energy current vs time

In [ ]:
# # Define the model function
# def model(t, a, b, c):
#     return a + b * np.exp(-c * t)

# current = np.diff(observable_expectation_value_vs_time_avg)/dt
# time_lst = np.arange(0, dt * number_of_data_points, dt)

# x = time_lst[:number_of_data_points-2]
# y = current

# initial_guess = [0.35, -0.4, 1.e-7]
# # Fit the data to the model
# popt, pcov = curve_fit(model, x, y, p0=initial_guess, maxfev=1000)

# # popt contains the optimized parameters a, b, and c
# a_fit, b_fit, c_fit = popt

# # Generate fitted values using the optimized parameters
# y_fit = model(x, *popt)
# print("Steady state energy current:", a_fit)

# fig, ax = plt.subplots(figsize=(8, 6))
# plt.scatter(x, y, label='Data', color='blue', s = 100)
# plt.plot(x, y_fit, label=f'Fit: $f(t) = {a_fit:.5f} + {b_fit:.3f}*exp(-{c_fit:.3f}*t)$', color='red', lw = 2)
# plt.xlabel(r'Time', labelpad = 20)
# plt.ylabel(r'Charge current', labelpad = 20)
# plt.legend()
# for spine in ax.spines.values():
#      spine.set_linewidth(2)
# ax.tick_params(width=2, length=10, direction='out', which='major')
# # plt.savefig("B3LYP_def2tzvp.png", dpi = 600, bbox_inches = 'tight')
# plt.show()

# Variational method for OPESme steady state

In [ ]:
def nonzero_params_from_circuit(obj):
    """
    Extract numeric non-zero parameters from:
      - a QuantumCircuit (obj.data)
      - a list of CircuitInstruction-like objects (has .operation.params)
    Returns list of floats in instruction order.
    """
    params = []
    def handle_param(p):
        try:
            v = float(p)
        except Exception:
            return None
        return v if v != 0.0 else None

    # QuantumCircuit (.data is list of (instr, qargs, cargs))
    if hasattr(obj, "data"):
        for instr, qargs, cargs in obj.data:
            for p in getattr(instr, "params", []):
                v = handle_param(p)
                if v is not None:
                    params.append(v)
        return params

    # Otherwise assume iterable of CircuitInstruction-like objects
    for instr in obj:
        op = getattr(instr, "operation", None) or getattr(instr, "instruction", None) or instr
        for p in getattr(op, "params", []):
            v = handle_param(p)
            if v is not None:
                params.append(v)
    return params

In [ ]:
def parameters_of_last_circuit(circuit_lst):
     all_params =  nonzero_params_from_circuit(circuit_lst[-1].evolved_state.data)
     # Grouping the parameters into sets of three (unitary, gamma_in, gamma_out)
     grouped_params = [all_params[i:i+3] for i in range(0, len(all_params), 3)]
     return grouped_params

trotter_angles = parameters_of_last_circuit(circuit_lst)
print("Trotter angles:", trotter_angles)

In [ ]:
statevector_lst[-1]

In [ ]:
angles, angle_history, energy_history = vqa.optimize(initial_angles = trotter_angles, learning_rate= 0.1, num_iterations = 25)

In [ ]:
np.save(f"energy_history_circuit_depth_{number_of_data_points - 1}.npy", np.array(energy_history))
np.save(f"angle_history_circuit_depth_{number_of_data_points - 1}.npy", np.array(angle_history))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
plt.plot(range(len(energy_history)), energy_history, marker = 'o', markersize = 15, lw = 2, color = 'rebeccapurple')
plt.xlabel(r'Iteration', labelpad = 20)
plt.ylabel(r'Energy', labelpad = 20)
for spine in ax.spines.values():
     spine.set_linewidth(2)
ax.tick_params(width=2, length=10, direction='in', which='major')
plt.show()

# Cluster optimization run data analysis 

In [ ]:
# path = "/Users/sasankadowarah/Ultimate_QM_MM/VQA/VQA_OPESme/cluster_data/solvated_OPESme_100"
path = "/Users/sasankadowarah/Ultimate_QM_MM/VQA/VQA_OPESme/cluster_data/OPESme_VQA_200"
os.chdir(path)

os.chdir(path +  f"/b0")
energy_history_1 = np.load(f"vacuum_active_second_q_depth_1_energy_history.npy")
os.chdir(path +  f"/b1")
energy_history_2 = np.load(f"vacuum_active_second_q_depth_2_energy_history.npy")
os.chdir(path +  f"/b2")
energy_history_3 = np.load(f"vacuum_active_second_q_depth_3_energy_history.npy")
os.chdir(path +  f"/b3")
energy_history_7 = np.load(f"vacuum_active_second_q_depth_7_energy_history.npy")
os.chdir(path +  f"/b4")
energy_history_8 = np.load(f"vacuum_active_second_q_depth_8_energy_history.npy")

In [ ]:
fig, ax = plt.subplots(figsize=(9, 7))
plt.plot(range(len(energy_history_1)), energy_history_1, marker = 'o', markersize = 10, lw = 2, label = 'Layer 1', color = 'rebeccapurple')
plt.plot(range(len(energy_history_2)), energy_history_2, marker = 'o', markersize = 10, lw = 2, label = 'Layer 2', color = 'darkorange')
plt.plot(range(len(energy_history_3)), energy_history_3, marker = 'o', markersize = 10, lw = 2, label = 'Layer 3', color = 'grey')
plt.plot(range(len(energy_history_7)), energy_history_7, marker = 'o', markersize = 10, lw = 2, label = 'Layer 7', color = 'orangered')
plt.plot(range(len(energy_history_8)), energy_history_8, marker = 'o', markersize = 10, lw = 2, label = 'Layer 8', color = 'dodgerblue')
plt.legend()
plt.xlabel(r'Iteration', labelpad = 10)
plt.ylabel(r'Energy', labelpad = 10)
for spine in ax.spines.values():
     spine.set_linewidth(2)
ax.tick_params(width=2, length=10, direction='in', which='major', color = "k")
# plt.axhline(y = -45)
path = "/Users/sasankadowarah/Ultimate_QM_MM/VQA/VQA_OPESme/cluster_data"
os.chdir(path)
plt.savefig("vacuum_OPESme_1_2_3_7_8_energy_convergence.png", dpi = 600, bbox_inches = 'tight')
plt.show()

In [ ]:
path = "/Users/sasankadowarah/Ultimate_QM_MM/VQA/VQA_OPESme/"
os.chdir(path)
with open("solvated_active_second_q.pickle", "rb") as f:
    solvated_fermionic_hamiltonian = pickle.load(f)
solvated_vqa = VQA(solvated_fermionic_hamiltonian, 1.e-10, True)

with open("vacuum_active_second_q.pickle", "rb") as f:
    vacuum_fermionic_hamiltonian = pickle.load(f)
vacuum_vqa = VQA(vacuum_fermionic_hamiltonian, 1.e-10, True)

In [ ]:
vacuum_OPESMe = vacuum_vqa.qubit_hamiltonian.to_matrix()
solvated_OPESMe = solvated_vqa.qubit_hamiltonian.to_matrix()

# Eignenvalues and eigenvectors
eigenvalues_vacuum, eigenvectors_vacuum = np.linalg.eigh(vacuum_OPESMe)
eigenvalues_solvated, eigenvectors_solvated = np.linalg.eigh(solvated_OPESMe)

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
plt.scatter(range(len(eigenvalues_solvated)), eigenvalues_solvated, label='Solvated', color='darkorange', s = 100, marker = 'o')
plt.scatter(range(len(eigenvalues_vacuum)), eigenvalues_vacuum, label='Vacuum', color='rebeccapurple', s = 50, marker = 'x')
plt.xlabel(r'Index', labelpad = 10)
plt.ylabel(r'Eigenvalues', labelpad = 10)
plt.legend()
for spine in ax.spines.values():
     spine.set_linewidth(2)
ax.tick_params(width=2, length=10, direction='in', which='major', color = "k")
# plt.savefig("vacuum_solvated_OPESme_spectrum.png", dpi = 600, bbox_inches = 'tight')
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Plot vacuum Hamiltonian
# im1 = ax1.imshow(np.real(vacuum_OPESMe) - np.real(solvated_OPESMe), cmap='RdBu')
im1 = ax1.imshow(np.real(solvated_OPESMe), cmap='RdBu')
ax1.set_title('Real part', pad=10)
ax1.set_xlabel('Index')
ax1.set_ylabel('Index')
plt.colorbar(im1, ax=ax1)

# Plot solvated Hamiltonian
# im2 = ax2.imshow(np.imag(vacuum_OPESMe) - np.imag(solvated_OPESMe), cmap='RdBu')
im2 = ax2.imshow(np.imag(solvated_OPESMe), cmap='RdBu')
ax2.set_title('Imaginary part', pad=10)
ax2.set_xlabel('Index')
ax2.set_ylabel('Index')
plt.colorbar(im2, ax=ax2)

# Adjust layout
plt.tight_layout()

# Set consistent styling
for ax in [ax1, ax2]:
     for spine in ax.spines.values():
          spine.set_linewidth(2)
     ax.tick_params(width=2, length=10, direction='in', which='major')
plt.show()